In [12]:
# %load nim.py
import math
import random
import time




In [39]:
def train(n):
    """
    Train an AI by playing `n` games against itself.
    """

    player = NimAI()

    # Play n games
    for i in range(n):
        if i % 1000 == 0:
            print(f"Playing training game {i + 1}")
        game = Nim()

        # Keep track of last move made by either player
        last = {
            0: {"state": None, "action": None},
            1: {"state": None, "action": None}
        }

        # Game loop
        while True:

            # Keep track of current state and action
            state = game.piles.copy()
            action = player.choose_action(game.piles)

            # Keep track of last state and action
            last[game.player]["state"] = state
            last[game.player]["action"] = action

            # Make move
            game.move(action)
            new_state = game.piles.copy()

            # When game is over, update Q values with rewards
            if game.winner is not None:
                # update input: old_state, action, new_state, reward
                # ??? What is the difference between these two lines???
                # updated twice = not updated???
                player.update(state, action, new_state, -1)
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    1
                )
                break

            # If game is continuing, no rewards yet
            elif last[game.player]["state"] is not None:
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    0
                )

    print("Done training")

    # Return the trained AI
    return player


def play(ai, human_player=None):
    """
    Play human game against the AI.
    `human_player` can be set to 0 or 1 to specify whether
    human player moves first or second.
    """

    # If no player order set, choose human's order randomly
    if human_player is None:
        human_player = random.randint(0, 1)

    # Create new game
    game = Nim()

    # Game loop
    while True:

        # Print contents of piles
        print()
        print("Piles:")
        for i, pile in enumerate(game.piles):
            print(f"Pile {i}: {pile}")
        print()

        # Compute available actions
        available_actions = Nim.available_actions(game.piles)
        time.sleep(1)

        # Let human make a move
        if game.player == human_player:
            print("Your Turn")
            while True:
                pile = int(input("Choose Pile: "))
                count = int(input("Choose Count: "))
                if (pile, count) in available_actions:
                    break
                print("Invalid move, try again.")

        # Have AI make a move
        else:
            print("AI's Turn")
            pile, count = ai.choose_action(game.piles, epsilon=False)
            print(f"AI chose to take {count} from pile {pile}.")

        # Make move
        game.move((pile, count))

        # Check for winner
        if game.winner is not None:
            print()
            print("GAME OVER")
            winner = "Human" if game.winner == human_player else "AI"
            print(f"Winner is {winner}")
            return


In [14]:
class Nim():

    def __init__(self, initial=[1, 3, 5, 7]):
        """
        Initialize game board.
        Each game board has
            - `piles`: a list of how many elements remain in each pile
            - `player`: 0 or 1 to indicate which player's turn
            - `winner`: None, 0, or 1 to indicate who the winner is
        """
        self.piles = initial.copy()
        self.player = 0
        self.winner = None

    @classmethod
    def available_actions(cls, piles):
        """
        Nim.available_actions(piles) takes a `piles` list as input
        and returns all of the available actions `(i, j)` in that state.

        Action `(i, j)` represents the action of removing `j` items
        from pile `i` (where piles are 0-indexed).
        """
        actions = set()
        for i, pile in enumerate(piles):
            for j in range(1, pile + 1):
                actions.add((i, j))
        return actions

    @classmethod
    def other_player(cls, player):
        """
        Nim.other_player(player) returns the player that is not
        `player`. Assumes `player` is either 0 or 1.
        """
        return 0 if player == 1 else 1

    def switch_player(self):
        """
        Switch the current player to the other player.
        """
        self.player = Nim.other_player(self.player)

    def move(self, action):
        """
        Make the move `action` for the current player.
        `action` must be a tuple `(i, j)`.
        """
        pile, count = action

        # Check for errors
        if self.winner is not None:
            raise Exception("Game already won")
        elif pile < 0 or pile >= len(self.piles):
            raise Exception("Invalid pile")
        elif count < 1 or count > self.piles[pile]:
            raise Exception("Invalid number of objects")

        # Update pile
        self.piles[pile] -= count
        self.switch_player()

        # Check for a winner
        if all(pile == 0 for pile in self.piles):
            self.winner = self.player

In [35]:
class NimAI():

    def __init__(self, alpha=0.5, epsilon=0.1):
        """
        Initialize AI with an empty Q-learning dictionary,
        an alpha (learning) rate, and an epsilon rate.

        The Q-learning dictionary maps `(state, action)`
        pairs to a Q-value (a number).
         - `state` is a tuple of remaining piles, e.g. (1, 1, 4, 4)
         - `action` is a tuple `(i, j)` for an action
        """
        self.q = dict()
        self.alpha = alpha
        self.epsilon = epsilon

    def update(self, old_state, action, new_state, reward):
        """
        Update Q-learning model, given an old state, an action taken
        in that state, a new resulting state, and the reward received
        from taking that action.
        """
        old = self.get_q_value(old_state, action)
        best_future = self.best_future_reward(new_state)
        self.update_q_value(old_state, action, old, reward, best_future)

    def get_q_value(self, state, action):
        """
        Return the Q-value for the state `state` and the action `action`.
        If no Q-value exists yet in `self.q`, return 0.
        """
        state = tuple(state)
        if (state, action) in self.q.keys():
            return self.q[(state, action)]
        else:
            return 0

    def update_q_value(self, state, action, old_q, reward, future_rewards):
        """
        Update the Q-value for the state `state` and the action `action`
        given the previous Q-value `old_q`, a current reward `reward`,
        and an estiamte of future rewards `future_rewards`.

        Use the formula:

        Q(s, a) <- old value estimate
                   + alpha * (new value estimate - old value estimate)

        where `old value estimate` is the previous Q-value,
        `alpha` is the learning rate, and `new value estimate`
        is the sum of the current reward and estimated future rewards.
        """
        state = tuple(state)
        self.q[(state, action)] = old_q + self.alpha * (reward + future_rewards - old_q)

        
    def best_future_reward(self, state):
        """
        Given a state `state`, consider all possible `(state, action)`
        pairs available in that state and return the maximum of all
        of their Q-values.

        Use 0 as the Q-value if a `(state, action)` pair has no
        Q-value in `self.q`. If there are no available actions in
        `state`, return 0.
        """
        actions = Nim.available_actions(state)
        state = tuple(state)
        if actions:
            best = 0
            for action in actions:
                if (state, action) in self.q.keys() and self.q[(state, action)] > best:
                    best = self.q[(state, action)]
            return best
        else:
            return 0

    def choose_action(self, state, epsilon=True):
        """
        Given a state `state`, return an action `(i, j)` to take.

        If `epsilon` is `False`, then return the best action
        available in the state (the one with the highest Q-value,
        using 0 for pairs that have no Q-values).

        If `epsilon` is `True`, then with probability
        `self.epsilon` choose a random available action,
        otherwise choose the best action available.

        If multiple actions have the same Q-value, any of those
        options is an acceptable return value.
        """
        actions = Nim.available_actions(state)
        if epsilon and random.random() < self.epsilon:
            return random.choice(list(actions)) # if error, change actions to list(actions)
        best = float("-inf")
        best_action = None
        unevaluated_action = None 
        state = tuple(state)
        
        for action in actions:
            if (state, action) in self.q.keys() and self.q[(state, action)] > best:
                best = self.q[(state, action)]
                best_action = action
            elif (state, action) not in self.q.keys():
                unevaluated_action = action
        
        if best_action is None:
            best_action = unevaluated_action
        if not best_action:
            print(best_action)
            print(unevaluated_action)
            print(state)
            print(actions)
            print(self.q)
        return best_action

In [40]:
# %load play.py
#from nim import train, play

ai = train(10000)
play(ai)


Playing training game 1
Playing training game 101
Playing training game 201
Playing training game 301
Playing training game 401
Playing training game 501
Playing training game 601
Playing training game 701
Playing training game 801
Playing training game 901
Playing training game 1001
Playing training game 1101
Playing training game 1201
Playing training game 1301
Playing training game 1401
Playing training game 1501
Playing training game 1601
Playing training game 1701
Playing training game 1801
Playing training game 1901
Playing training game 2001
Playing training game 2101
Playing training game 2201
Playing training game 2301
Playing training game 2401
Playing training game 2501
Playing training game 2601
Playing training game 2701
Playing training game 2801
Playing training game 2901
Playing training game 3001
Playing training game 3101
Playing training game 3201
Playing training game 3301
Playing training game 3401
Playing training game 3501
Playing training game 3601
Playing train

Choose Pile:  2
Choose Count:  2



Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 3
Pile 3: 4

AI's Turn
AI chose to take 3 from pile 3.

Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 3
Pile 3: 1

Your Turn


Choose Pile:  1
Choose Count:  1



Piles:
Pile 0: 1
Pile 1: 2
Pile 2: 3
Pile 3: 1

AI's Turn
AI chose to take 1 from pile 0.

Piles:
Pile 0: 0
Pile 1: 2
Pile 2: 3
Pile 3: 1

Your Turn


...


Piles:
Pile 0: 0
Pile 1: 2
Pile 2: 2
Pile 3: 1

AI's Turn
AI chose to take 1 from pile 3.

Piles:
Pile 0: 0
Pile 1: 2
Pile 2: 2
Pile 3: 0

Your Turn


Choose Pile:  1
Choose Count:  1



Piles:
Pile 0: 0
Pile 1: 1
Pile 2: 2
Pile 3: 0

AI's Turn
AI chose to take 2 from pile 2.

Piles:
Pile 0: 0
Pile 1: 1
Pile 2: 0
Pile 3: 0

Your Turn


Choose Pile:  1
Choose Count:  1



GAME OVER
Winner is AI


In [41]:
ai_toy = train(200)
play(ai_toy)

Playing training game 1
Playing training game 101
Done training

Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 5
Pile 3: 7

Your Turn


Choose Pile:  3
Choose Count:  3



Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 5
Pile 3: 4

AI's Turn
AI chose to take 3 from pile 1.

Piles:
Pile 0: 1
Pile 1: 0
Pile 2: 5
Pile 3: 4

Your Turn


Choose Pile:  3
Choose Count:  1



Piles:
Pile 0: 1
Pile 1: 0
Pile 2: 5
Pile 3: 3

AI's Turn
AI chose to take 5 from pile 2.

Piles:
Pile 0: 1
Pile 1: 0
Pile 2: 0
Pile 3: 3

Your Turn


Choose Pile:  3
Choose Count:  3



Piles:
Pile 0: 1
Pile 1: 0
Pile 2: 0
Pile 3: 0

AI's Turn
AI chose to take 1 from pile 0.

GAME OVER
Winner is Human


In [48]:
play(ai, human_player=0)


Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 5
Pile 3: 7

Your Turn


Choose Pile:  3
Choose Count:  3



Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 5
Pile 3: 4

AI's Turn
AI chose to take 1 from pile 2.

Piles:
Pile 0: 1
Pile 1: 3
Pile 2: 4
Pile 3: 4

Your Turn


Choose Pile:  1
Choose Count:  2



Piles:
Pile 0: 1
Pile 1: 1
Pile 2: 4
Pile 3: 4

AI's Turn
AI chose to take 1 from pile 2.

Piles:
Pile 0: 1
Pile 1: 1
Pile 2: 3
Pile 3: 4

Your Turn


...


Piles:
Pile 0: 0
Pile 1: 1
Pile 2: 3
Pile 3: 1

AI's Turn
AI chose to take 2 from pile 2.

Piles:
Pile 0: 0
Pile 1: 1
Pile 2: 1
Pile 3: 1

Your Turn


Choose Pile:  2
Choose Count:  1



Piles:
Pile 0: 0
Pile 1: 1
Pile 2: 0
Pile 3: 1

AI's Turn
AI chose to take 1 from pile 1.

Piles:
Pile 0: 0
Pile 1: 0
Pile 2: 0
Pile 3: 1

Your Turn


Choose Pile:  3
Choose Count:  1



GAME OVER
Winner is AI
